In [11]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('merged_Dump.csv')

/var/folders/s8/5rrw77y553vc4scyrbx8sdg40000gn/T/ipykernel_36852/2305914882.py:4: DtypeWarning: Columns (22,23,24,25,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,90,91,92,98,100,108,111,113,114,115,116,117,118,119,120,121,122,123,128,137,138,139,141,142,143,144,145,146,147,151,152,153,154) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('merged_Dump.csv')


In [16]:
slice_df = df[df['protocolSection.identificationModule.nctId'] == 'NCT05705401']
slice_df.to_csv('filtered_data_NCT05705401.csv', index=False)

In [30]:
#1. completion date between 2008.01-2021.12.31
df['completion_date'] = pd.to_datetime(df['protocolSection.statusModule.completionDateStruct.date'], errors='coerce', infer_datetime_format=True)

# set the range of date
start_date = '2008-01-01'
end_date = '2021-12-31'

#2. Have start data
df['start_date'] = pd.to_datetime(df['protocolSection.statusModule.startDateStruct.date'], errors='coerce', infer_datetime_format=True)

#3. Have primary start date
df['primary_start_date'] = pd.to_datetime(df['protocolSection.statusModule.primaryCompletionDateStruct.date'], errors='coerce', infer_datetime_format=True)

#4. Exclude Phase 1 and 4
def exclude_phases(phases):
    if isinstance(phases, list):
        return not any(phase in ['PHASE1', 'PHASE4'] for phase in phases)
    return True

#5. Sponsor contains oncology OR trials reported results at Oncology
# protocolSection.sponsorCollaboratorsModule.leadSponsor.name
df['sponsor'] = df['protocolSection.identificationModule.organization.fullName']

#6. Recruitment Status is Completed
df['Recruitment'] = df['protocolSection.statusModule.overallStatus']

#7. studyType is Interventional
df['Interventional']= df['protocolSection.designModule.studyType']

filtered_df = df[(df['completion_date'] >= start_date) & 
                 (df['completion_date'] <= end_date) &
                 (df['start_date'].notna()) &
                 (df['primary_start_date'].notna()) &
                 (df['protocolSection.designModule.phases'].apply(exclude_phases)) &
                 (df['sponsor'].str.contains('oncology', case=False, na=False)) &
                 (df['Recruitment']=='COMPLETED') &
                 (df['Interventional']=='INTERVENTIONAL')]

# print the number of filtered data
count = filtered_df.shape[0]
print(f"The number of data from January 2008 to December 31th 2021: {count}")

/var/folders/s8/5rrw77y553vc4scyrbx8sdg40000gn/T/ipykernel_36852/1895934556.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['completion_date'] = pd.to_datetime(df['protocolSection.statusModule.completionDateStruct.date'], errors='coerce', infer_datetime_format=True)
/var/folders/s8/5rrw77y553vc4scyrbx8sdg40000gn/T/ipykernel_36852/1895934556.py:9: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['start_date'] = pd.to_datetime(df['protocolSection.statusModule.startDateStruct.date'], errors='coerce', infer_datetime_format=True)
/var/folders/s8/5rrw77y5

2008년 1월부터 2021년 12월 31일까지의 데이터 개수: 539
